In [3]:
%matplotlib ipympl
from impedance.preprocessing import readBioLogic
from impedance.models.circuits import CustomCircuit
import matplotlib.pyplot as plt
import numpy as np
from models import *
import mpmath as mp
import os
import csv
import pandas as pd
import scipy

In [2]:

# df_bks=pd.read_excel(os.path.join("..","exp data","cell setup info","Cell table stacked electrodes_BKSComments.xlsx"))
# df_bks=df_bks.dropna(subset=["Name ","Datum"])
# df_bks=df_bks.loc[df_bks["Datum"]!=float(19082018.0)]
# df_bks=df_bks.loc[df_bks["Name "]!="RM_stacked cells second DS_3+12_1"]
# df_bks

In [3]:
@element(num_params=6, units=["Ohm", "Ohm F", "Ohm F","","",""],overwrite=True)
def TLMtwoQvary(p,f):
   
    omega = 2 * np.pi * np.array(f)
    R1,R1R2, R1Q1, R1Q2, delta1, alpha = p[0], p[1], p[2], p[3], p[4], p[5]
    delta2=1-delta1
    R2=R1R2/R1
    R2Q2=R1Q2/R1*R2
    k12=R2/R1
    wc1=1/(R1Q1)
    wc2=1/(R2Q2)
    S1=np.sqrt((omega*1j)**alpha/wc1)
    S2=np.sqrt((omega*1j)**alpha/wc2)
    C1=1/np.tanh(delta1*S1)
    C2=1/np.tanh(delta2*S2)

    Z=2*(R1)*(C1*C2*S1*k12+S2)/(S1*(C2*S1*k12+C1*S2))

    return Z

In [ ]:
def find_index_of_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def lin_fun(x,a,b):
    return a*(x+b)

def find_turning_freq(f,z):
    x=np.real(z)[-6:]
    y=-np.imag(z)[-6:]
    popt, _ = scipy.optimize.curve_fit(lin_fun,x,y)
    x_intercept=popt[1]
    idx=find_index_of_nearest(np.imag(z),x_intercept)
    freq_turn=f[idx]
    return freq_turn,idx,popt[0],popt[1]

In [ ]:
folder_path=os.path.join("..","exp data","EIS mpt files","two combined")
file_path=os.path.join(folder_path,"RM_stacked second run DS_3+12_2_C06.mpt")
f,z=readBioLogic(file_path)
f_turn,idx,a,b=find_turning_freq(f,z)
print(f_turn,idx)
fig, ax=plt.subplots()
ax.plot(np.real(z),-np.imag(z),"*")
ax.plot(np.real(z[idx]),-np.imag(z[idx]),"*")
xx=np.linspace(0,3000)
yy=lin_fun(xx,a,b)
ax.plot(xx,yy)
ax.set_aspect("equal")
print(a,b)
idx_l=find_index_of_nearest(f_turn/10,f)
ax.plot(np.real(z[idx_l]),-np.imag(z[idx_l]),"*")

In [ ]:
folder_path=os.path.join("..","exp data","EIS mpt files","two combined")
first_itr=True
for file_name in os.listdir(folder_path):
    file_path=os.path.join(folder_path,file_name)
    f,z=readBioLogic(file_path)
    
    f_orig=f
    z_orig=z

    f_turn,idx,_,_=find_turning_freq(f,z)
    idx_l=find_index_of_nearest(f_turn/10,f)

    f=f[0:idx_l+1]
    z=z[0:idx_l+1]

    Rsep=100
    R1=2000
    R2=5000
    Q1=1e-4
    Q2=3e-4
    delta=0.18
    alpha=1
    c=CustomCircuit(initial_guess=[Rsep,R1,R1*R2,R1*Q1,R2*Q2,delta,alpha],circuit="R_1-TLMtwoQvary")

    mp.dps=500
    s_mod=np.hstack((np.abs(z),np.abs(z)))
    c.fit(impedance=z,frequencies=f,sigma=s_mod,bounds=((0,0,0,0,0,0.167,0.9),(np.inf,np.inf,np.inf,np.inf,np.inf,0.6,1)))
    # c.fit(impedance=z,frequencies=f,bounds=((0,0,0,0,0,0.167,0.9),(np.inf,np.inf,np.inf,np.inf,np.inf,0.6,1)))
    
    z_fitted=c.predict(frequencies=f)
    fcsv="../fitted parameters/two combined_mod_fit.csv"
    # fcsv="../fitted parameters/two combined.csv"
    
    fig, ax= plt.subplots()
    ax.plot(np.real(z_fitted),-np.imag(z_fitted),linewidth=1)
    ax.plot(np.real(z),-np.imag(z),linewidth=1)
    ax.set_title(file_name)
    ax.set_aspect("equal")
    fig.savefig(f"../fitted plots/two combined/{file_name[:-4]}",dpi=700)

    if first_itr:
        data=[
            ["filename","Rsep","R1","R1R2","R1Q1","R2Q2","delta","α"],
            [f"{file_name}",c.parameters_[0],c.parameters_[1],c.parameters_[2],c.parameters_[3],c.parameters_[4],c.parameters_[5],c.parameters_[6]]
        ]
        first_itr=False
        with open(fcsv,"w",newline="",encoding='UTF-8') as csv_file:
            writer=csv.writer(csv_file)
            writer.writerows(data)
    else:
        data=[
            [f"{file_name}",c.parameters_[0],c.parameters_[1],c.parameters_[2],c.parameters_[3],c.parameters_[4],c.parameters_[5],c.parameters_[6]]
        ]
        with open(fcsv,"a",newline="",encoding='UTF-8') as csv_file:
            writer=csv.writer(csv_file)
            writer.writerows(data)

In [5]:

df_bks=pd.read_excel(os.path.join("..","exp data","cell setup info","Cell table stacked electrodes_BKSComments.xlsx"))
df_bks=df_bks.dropna(subset=["Name ","Datum"])
df_bks=df_bks.loc[df_bks["Datum"]!=float(19082018.0)]
# df_bks=df_bks.loc[df_bks["Name "]!="RM_stacked cells second DS_3+12_1"]
df_bks
# ff=df_bks["Name "]
# ff!="RM_stacked cells second DS_3+12_1"



,Name,Datum,Project,m_electrode [mg],m_cc [mg],M_coating [mg],Porosity,Elyte conduct. [mS/cm],Diameter [cm],electr. thickn. [µm],cc thickness [µm],coating thickness [µm],2* Rion [Ohm] (aus EC-Lab),tau,McMullin,Comment,Masse Ref coating,Abtrag [%]
24,RM_stacked cells second S3_1,20180903.0,Stacking electrodes,26.480,0.0,NaN,0.53,0.353,1.095,268.0,0.0,NaN,986.0,NaN,NaN,NaN,NaN,NaN
26,RM_stacked cells second S3_2,20180903.0,Stacking electrodes,27.060,0.0,NaN,0.52,0.353,1.095,269.0,0.0,NaN,1014.0,NaN,NaN,NaN,NaN,NaN
28,RM_stacked cells second S12_1,20180903.0,Stacking electrodes,6.410,0.0,NaN,0.41,0.353,1.095,52.0,0.0,NaN,417.0,NaN,NaN,NaN,NaN,NaN
30,RM_stacked cells second S12_2,20180903.0,Stacking electrodes,6.495,0.0,NaN,0.43,0.353,1.095,54.0,0.0,NaN,402.0,NaN,NaN,NaN,NaN,NaN
32,RM_stacked cells second D3_1,20180903.0,Stacking electrodes,NaN,0.0,NaN,0.52,0.353,1.095,580.0,0.0,NaN,2217.0,NaN,NaN,NaN,NaN,NaN
34,RM_stacked cells second D3_2,20180903.0,Stacking electrodes,NaN,0.0,NaN,0.52,0.353,1.095,593.5,0.0,NaN,2133.0,NaN,NaN,NaN,NaN,NaN
36,RM_stacked cells second D12_1,20180903.0,Stacking electrodes,NaN,0.0,NaN,0.41,0.353,1.095,106.0,0.0,NaN,912.0,NaN,NaN,NaN,NaN,NaN
38,RM_stacked cells second D12_2,20180903.0,Stacking electrodes,NaN,0.0,NaN,0.43,0.353,1.095,112.0,0.0,NaN,972.0,NaN,NaN,NaN,NaN,NaN
40,RM_stacked cells second DS_3+12_1,20180903.0,Stacking electrodes,NaN,0.0,NaN,NaN,0.353,1.095,329.0,0.0,NaN,1968.0,NaN,NaN,NaN,NaN,NaN
42,RM_stacked cells second DS_3+12_2,20180903.0,Stacking electrodes,NaN,0.0,NaN,NaN,0.353,1.095,334.0,0.0,NaN,1923.0,NaN,NaN,NaN,NaN,NaN
